### Getting player data from Logs.tf API

In [8]:
import json
import pandas as pd
import requests

In [9]:
url = 'https://logs.tf/api/v1/log?player=76561198049461668&limit=10'
response = requests.get(url, verify=False)
game_list = json.loads(response.content)

/home/reedoken/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'logs.tf'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [10]:
df = pd.DataFrame(columns=['player_id', 'team', 'kills', 'player_class', 'assists', 'dmg', 'deaths'])
default_url = 'https://logs.tf/api/v1/log/'

In [11]:
for i in game_list['logs']:
    if (i['players'] != 12):
        continue
    game_id = (i['id'])
    url = default_url + str(game_id)
    response = requests.get(url, verify=False)
    stats = json.loads(response.content)
    for j in stats['players']:
        player_id = j
        team = stats['players'][j]['team']
        kills = stats['players'][j]['kills']
        player_class = stats['players'][j]['class_stats'][0]['type']
        assists = stats['players'][j]['assists']
        dmg = stats['players'][j]['dmg']
        deaths = stats['players'][j]['deaths']

        row_dict = {'player_id': player_id, 'team': team, 'kills': kills, 'player_class': player_class, 'assists': assists, 'dmg': dmg, 'deaths': deaths}
        temp_df = pd.DataFrame([row_dict])
        df = pd.concat([df, temp_df], ignore_index=True)

/home/reedoken/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'logs.tf'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/reedoken/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'logs.tf'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/reedoken/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'logs.tf'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/reedoken/.local/lib/python3.10/site-packages/urllib3

In [12]:
df.head()

,player_id,team,kills,player_class,assists,dmg,deaths
0,[U:1:75438474],Blue,8,scout,6,2420,8
1,[U:1:85413659],Blue,7,soldier,5,2609,10
2,[U:1:10403381],Blue,23,scout,13,4539,0
3,[U:1:117668615],Red,3,soldier,1,1786,12
4,[U:1:122797343],Blue,12,demoman,6,4814,2


In [19]:
df['player_id'] = df['player_id'].str.rstrip()

In [27]:
from steam.steamid import SteamID

def get_steamid(player_id):
    idnum = SteamID(player_id).as_64
    return idnum

In [49]:
df['steamid'] = df['player_id'].apply(lambda x: str(SteamID(x).as_64))

In [50]:
df['steamid'] = df['steamid'].str.rstrip()

In [51]:
df.head()

,player_id,team,kills,player_class,assists,dmg,deaths,steamid
0,[U:1:75438474],Blue,8,scout,6,2420,8,76561198035704202
1,[U:1:85413659],Blue,7,soldier,5,2609,10,76561198045679387
2,[U:1:10403381],Blue,23,scout,13,4539,0,76561197970669109
3,[U:1:117668615],Red,3,soldier,1,1786,12,76561198077934343
4,[U:1:122797343],Blue,12,demoman,6,4814,2,76561198083063071


### Managing player aliases from RGL API

In [85]:
df.head()

,player_id,team,kills,player_class,assists,dmg,deaths,steamid
0,[U:1:75438474],Blue,8,scout,6,2420,8,76561198035704202
1,[U:1:85413659],Blue,7,soldier,5,2609,10,76561198045679387
2,[U:1:10403381],Blue,23,scout,13,4539,0,76561197970669109
3,[U:1:117668615],Red,3,soldier,1,1786,12,76561198077934343
4,[U:1:122797343],Blue,12,demoman,6,4814,2,76561198083063071


In [88]:
columns = df.drop(['player_id', 'steamid'], axis=1).columns.tolist()

player_df = df.drop(columns, axis=1)
player_df = player_df.drop_duplicates()

player_df = player_df.drop(index=player_df.tail(10).index)

steamids = [f'"{steamid}"' for steamid in player_df['steamid']]

# create a string with comma-separated SteamIDs enclosed in square brackets
steamids_str = "[" + ", ".join(steamids) + "]"

# create the JSON data to be sent in the POST request
data = json.loads(steamids_str)

print(data)


['76561198035704202', '76561198045679387', '76561197970669109', '76561198077934343', '76561198083063071', '76561198017539604', '76561198045838834', '76561198146255593', '76561198068686187', '76561198051499968', '76561198216487540', '76561198049461668', '76561198053743602', '76561198044219900', '76561198843277640', '76561197995794137', '76561198090577320', '76561198044736807', '76561198099843841', '76561198067295068', '76561198118877005', '76561198094788852', '76561198084582998', '76561198096347625', '76561198079513105']


In [84]:
rgl_player_url = 'https://api.rgl.gg/v0/profile/getmany'

headers = {
    "accept": "*/*",
    "Content-Type": "application/json",
}


response = requests.post(url, headers=headers, data=data)

print(response)

print(response.json())

<Response [405]>


JSONDecodeError: Expecting value: line 1 column 1 (char 0)